# Proximal algorithms

In this notebook, we code our proximal optimization algorithms.

# 1. Proximal Gradient algorithm

For minimizing a function $F:\mathbb{R}^n \to \mathbb{R}$ equal to $f+g$ where $f$ is differentiable and the $\mathbf{prox}$ of $g$ is known, given:
* the function to minimize `F`
* a 1st order oracle for $f$ `f_grad` 
* a proximity operator for $g$ `g_prox` 
* an initialization point `x0`
* the sought precision `PREC` 
* a maximal number of iterations `ITE_MAX` 
* a display boolean variable `PRINT` 

these algorithms perform iterations of the form
$$ x_{k+1} = \mathbf{prox}_{\gamma g}\left( x_k - \gamma \nabla f(x_k) \right) $$
where $\gamma$ is a stepsize to choose.



> Fill the function below with the proximal gradient algorithm.

> How would you implement the precision stopping criteria.

In [1]:
import numpy as np
import timeit

def proximal_gradient_algorithm(F , f_grad , g_prox , x0 , step , PREC , ITE_MAX , PRINT ):
    x = np.copy(x0)
    x_tab = np.copy(x)
    if PRINT:
        print("------------------------------------\n Proximal gradient algorithm\n------------------------------------\nSTART    -- stepsize = {:0}".format(step))
    t_s =  timeit.default_timer()
    for k in range(ITE_MAX):
        x_prev = x
        g = f_grad(x)
        x = g_prox(x - step*g , step)  #######  ITERATION

        x_tab = np.vstack((x_tab,x))

        # Stop Condition
        if np.abs(np.linalg.norm(x)-np.linalg.norm(x_prev)) < PREC:
            break

    t_e =  timeit.default_timer()
    if PRINT:
        print("FINISHED -- {:d} iterations / {:.6f}s -- final value: {:f}\n\n".format(k,t_e-t_s,F(x)))
    return x,x_tab

# Stochastic gradient 


In the following, instead of considering $f$ as a whole, we will use its structure 
$$ f(x) := \frac{1}{m}\sum_{i=1}^m f_i(x)$$

> Implement the gradient related to $f_i$, related to one example, in ``[logistic_regression_student]``

With this structure a popular minimization algorithm is the *stochastic gradient algorithm* which writes as follows:
* Select uniformly $i$ in $1,..,m$
* $x_{k+1} = \mathbf{prox}_{\gamma g}\left( x_k - \gamma_k \nabla f_i(x_k) \right) $

> Implement this algorithm with a stepsize vanishing as $1/k$

In [2]:
import random

def stochastic_gradient_descent(F , f_grad , f_grad_i , g_prox , x0 , m, step , PREC , ITE_MAX , PRINT ):
    x = np.copy(x0)
    x_tab = np.copy(x)
    if PRINT:
        print("------------------------------------\n Stochastic gradient descent algorithm\n------------------------------------\nSTART    -- stepsize = {:0}".format(step))
    t_s =  timeit.default_timer()
    for k in range(ITE_MAX):
        x_prev = x
        
        gamma = (step/(k+1))
        
        i = np.random.randint(1,m) # choose uniformly i in 1, ..., nbr of realisation = final_grades.size
        
        x = x - gamma * f_grad_i(x,i)

        x_tab = np.vstack((x_tab,x))

        ## Stop Condition 
        #if np.abs(np.linalg.norm(x)-np.linalg.norm(x_prev)) < PREC:
        #    break
        
        # Cesaro stop criterion
        if np.allclose(x, cesaro(x_tab[:-1]), rtol=PREC):
            break

    t_e =  timeit.default_timer()
    if PRINT:
        print("FINISHED -- {:d} iterations / {:.6f}s -- final value: {:f}\n\n".format(k,t_e-t_s,F(x)))
    return x,x_tab

In [3]:
def saga(F , f_grad , f_grad_i , g_prox , x0 , m, step , PREC , ITE_MAX ,  PRINT):
    x = np.copy(x0)
    x_tab = np.copy(x)
    fi_tab = np.copy(f_grad_i)
    phi = x0
    
    if PRINT:
        print("------------------------------------\n SAGA \n------------------------------------\nSTART    -- stepsize = {:0}".format(step))
    t_s =  timeit.default_timer()
    for k in range(ITE_MAX):
        x_prev = x
        fi_prev = f_grad_i(x_prev)
        
        gamma = (step/(k+1))
        
        i = np.random.randint(1,m) # choose uniformly i in 1, ..., nbr of realisation = final_grades.size
        
        print("a")
        
        w = x - gamma * (f_grad_i(x,i) - fi_prev + cesaro(fi_tab))
                         
        print("a")
                         
        x = prox(x, w, F, gamma)
        
        x_tab = np.vstack((x_tab,x))
        fi_tab = np.vstack((fi_tab,f_grad_i(x)))
        
        # Cesaro stop criterion
        if np.allclose(x, cesaro(x_tab[:-1]), rtol=PREC):
            break
        
    t_e =  timeit.default_timer()
    if PRINT:
        print("FINISHED -- {:d} iterations / {:.6f}s -- final value: {:f}\n\n".format(k,t_e-t_s,F(x)))
    return x,x_tab

## Tools

### Cesaro mean

In [4]:
def cesaro(X):
    m = np.shape(X)[0] # number of iterates
    
    return np.sum(X,axis=0)/m

### Proximal operator

In [5]:
def prox(x, y, h, gamma):
    return (np.argmin(h(x) + np.linalg.norm(x-y)^2)/(2*gamma))